<h1 align="center">T20 World Cup Cricket Data Pre Processing<h1/>

In [68]:
# Install a pip pandas package in the current Jupyter kernel
import sys

#Basic Packages related to Data Analysis
!{sys.executable} -m pip install pandas


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [69]:
#import necessary libraries
import pandas as pd 
import json

<h3 style="color:blue">(1) Process Match Results<h3/>

In [70]:
#Read Match Results file
with open('/D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_json_files/t20_wc_match_results.json') as f:
     data = json.load(f)  

df_match = pd.DataFrame(data[0]['matchSummary'])

#Trim additional spaces on string columns
df_match = df_match.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Validation of match information
df_match.head()

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [71]:
#Check the total no of rows and columns
df_match.shape

(45, 7)

**Use scorecard as a match id to link with other tables**

In [72]:
#Rename columns
df_match.rename({'scorecard':'match_id'}, axis = 1, inplace = True)

#Verify if the column has been renamed successfully
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   team1      45 non-null     object
 1   team2      45 non-null     object
 2   winner     45 non-null     object
 3   margin     45 non-null     object
 4   ground     45 non-null     object
 5   matchDate  45 non-null     object
 6   match_id   45 non-null     object
dtypes: object(7)
memory usage: 2.6+ KB


In [73]:
#Check for any null values in that data for all columns
df_match.isnull().sum().sort_values(ascending=False)

team1        0
team2        0
winner       0
margin       0
ground       0
matchDate    0
match_id     0
dtype: int64

In [74]:
#Null Condition Check in required non_decimal columns

# List columns to check for null values
columns_to_check = ['team1', 'team2', 'matchDate', 'match_id']

# Identification of rows with null values
rows = df_match[columns_to_check].isna().any(axis=1) | (df_match[columns_to_check] == '').any(axis=1)

# Drop null rows from the original DataFrame
df_match = df_match[~rows].reset_index(drop=True)

In [75]:
#Deduplication

df_match.drop_duplicates(subset=['team1', 'team2', 'matchDate', 'match_id'], keep='first', inplace=True)

In [76]:
#Validation of Unique_id column

pattern = r'^T20I # \d+$'

#Filter and consider only T20 records
df_match = df_match[df_match['match_id'].str.match(pattern)]

In [77]:
#store final data as csv file for validation
df_match.to_csv('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_csv_files/match_summary.csv', index = False)

**Create a match ids dictionary that maps team names to a unique match id. This will be useful later on to link with other dataframes**

In [78]:
#Create a unique key attribute combination to identify each record
match_ids_dict = {}

for index, row in df_match.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + ' Vs ' + row['team1']
    
    match_ids_dict[key1] = row ['match_id']
    match_ids_dict[key2] = row ['match_id']

#Validation of identifier records
match_ids_dict

{'Namibia Vs Sri Lanka': 'T20I # 1823',
 'Sri Lanka Vs Namibia': 'T20I # 1823',
 'Netherlands Vs U.A.E.': 'T20I # 1825',
 'U.A.E. Vs Netherlands': 'T20I # 1825',
 'Scotland Vs West Indies': 'T20I # 1826',
 'West Indies Vs Scotland': 'T20I # 1826',
 'Ireland Vs Zimbabwe': 'T20I # 1828',
 'Zimbabwe Vs Ireland': 'T20I # 1828',
 'Namibia Vs Netherlands': 'T20I # 1830',
 'Netherlands Vs Namibia': 'T20I # 1830',
 'Sri Lanka Vs U.A.E.': 'T20I # 1832',
 'U.A.E. Vs Sri Lanka': 'T20I # 1832',
 'Ireland Vs Scotland': 'T20I # 1833',
 'Scotland Vs Ireland': 'T20I # 1833',
 'West Indies Vs Zimbabwe': 'T20I # 1834',
 'Zimbabwe Vs West Indies': 'T20I # 1834',
 'Netherlands Vs Sri Lanka': 'T20I # 1835',
 'Sri Lanka Vs Netherlands': 'T20I # 1835',
 'Namibia Vs U.A.E.': 'T20I # 1836',
 'U.A.E. Vs Namibia': 'T20I # 1836',
 'Ireland Vs West Indies': 'T20I # 1837',
 'West Indies Vs Ireland': 'T20I # 1837',
 'Scotland Vs Zimbabwe': 'T20I # 1838',
 'Zimbabwe Vs Scotland': 'T20I # 1838',
 'Australia Vs New Zea

<h4 style="color:blue">(2) Process Batting Summary</h4>

In [79]:
#Read Batting Summary file

with open('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_json_files/t20_wc_batting_summary.json') as f:
     data = json.load(f)
    
     #Initialize an empty list to store all records
     all_records = []
    
     for rec in data:
            all_records.extend(rec['battingSummary'])

#Convert it to Dataframe
df_batting = pd.DataFrame(all_records)

#Validation of batting information
df_batting.head()

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c †Mendis b Karunaratne,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33


In [80]:
#Check the total no of rows and columns
df_batting.shape

(699, 10)

In [81]:
#Trim additional spaces on string columns
df_batting = df_batting.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [82]:
#Addition of new columns
df_batting['out/not_out'] = df_batting.dismissal.apply(lambda x: "out" if len(x)>0 else "not_out")
df_batting['match_id'] = df_batting['match'].map(match_ids_dict)

In [83]:
#Drop unwanted columns
df_batting.drop(columns=['dismissal'], inplace = True)

#Validation of batting information
df_batting.head(11)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out,T20I # 1823
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,out,T20I # 1823
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,out,T20I # 1823
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,not_out,T20I # 1823
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,out,T20I # 1823
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis†,6,6,0,0,100.00,out,T20I # 1823


In [84]:
# Drop null value rows based on match_id
df_batting.dropna(subset=['match_id'], inplace=True)

**Remove UTF-8 characters**

In [85]:
import re

df_batting['batsmanName'] = df_batting['batsmanName'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))

#Validation of batsman information
df_batting.head(11)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out,T20I # 1823
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,out,T20I # 1823
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,out,T20I # 1823
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,not_out,T20I # 1823
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,out,T20I # 1823
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis,6,6,0,0,100.00,out,T20I # 1823


In [86]:
#store final data as csv file for validation
df_batting.to_csv('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_csv_files/batting_summary.csv', index = False)

<h4 style="color:blue">(3) Process Bowling Summary</h4>

In [87]:
#Read Bowling Summary file
with open('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_json_files/t20_wc_bowling_summary.json') as f:
    data = json.load(f)
    
    #Initialize an empty list to store all records
    all_records = []
    
    for rec in data:
        all_records.extend(rec['bowlingSummary'])

#Comnvert it to DataFrame
df_bowling = pd.DataFrame(all_records)

#Validation of bowling information
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0


In [88]:
#Check the total no of rows and columns
df_bowling.shape

(500, 13)

In [89]:
#Addition of new column
df_bowling['match_id'] = df_bowling['match'].map(match_ids_dict)

#Validation of bowling information
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823


In [90]:
# Drop null value rows based on match_id
df_bowling.dropna(subset=['match_id'], inplace=True)

In [91]:
df_bowling.to_csv('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_csv_files/bowling_summary.csv', index = False)

<h4 style="color:blue">(4) Process Players Information</h4>

In [92]:
#Read Players Information file

with open('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_json_files/t20_wc_player_info.json') as f:
    data = json.load(f)
    
#Convert it to DataFrame
df_players = pd.DataFrame(data)

#Validation of Players Information file
df_players.head(10)

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder,
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder,David Wiese joined a marked outflow of South A...
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder,
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter,
9,Kusal Mendis†,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter,"Blessed with a compact technique, an aggressiv..."


In [93]:
#Check the total no of rows and columns
df_players.shape

(219, 6)

**Remove UTF-8 characters**

In [94]:
import re

df_players['name'] = df_players['name'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))

#Validation of batsman information
df_players.head(11)

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder,
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder,David Wiese joined a marked outflow of South A...
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder,
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter,
9,Kusal Mendis,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter,"Blessed with a compact technique, an aggressiv..."


In [95]:
#Check if there are any duplicates on name attribute
duplicates = df_players[df_players.duplicated('name')]

print(duplicates)

Empty DataFrame
Columns: [name, team, battingStyle, bowlingStyle, playingRole, description]
Index: []


In [96]:
#Drop records if there are any duplicates
df_players.drop_duplicates('name', inplace=True)

In [97]:
df_players.to_csv('D:/UE_College_study/Sem1/Data analytics/Final_project/DA-final project/t20_csv_files/players_no_images.csv', index = False)